# Unikey# 490166637 
COMP5046 Assignment 1
*Make sure you change the file name with your unikey.*

# Readme
*If there is something to be noted for the user, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style*

***Visualising the comparison of different results is a good way to justify your decision.***

# 1 - Data Preprocessing (Personality chat datasets)

## 1.1. Download Dataset (Personality chat datasets)

In [1]:
import nltk
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import tensorflow as tf


import gzip
import gensim 
from gensim.models.callbacks import CallbackAny2Vec
from scipy import spatial
from collections import OrderedDict
from google.colab import files
# from google.colab import drive
# drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
# Please comment your code
# Please comment your code

# Code to download personality files into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1T-HcbBOf98KwUZcLuvbFUOepLqZhVHA-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('qna_chitchat_the_comic.tsv') 

id = '11ykY78Tsagf9BAH_R2EgrAmz02iTLCw0'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('qna_chitchat_the_friend.tsv') 

id = '1stmouBrMsIoyDgPN5RhVy4qARwAJ3jhg'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('qna_chitchat_the_professional.tsv')

    100% |████████████████████████████████| 993kB 21.2MB/s 


In [0]:
df_prof = pd.read_csv('qna_chitchat_the_professional.tsv', sep="\t")
df_comic = pd.read_csv('qna_chitchat_the_comic.tsv', sep="\t")
df_friend = pd.read_csv('qna_chitchat_the_friend.tsv', sep="\t")

## 1.2. Preprocess data (Personality chat datasets)

*You are required to describe which data preprocessing techniques were conducted with justification of your decision. *

Personality chat datasets have lot of contractions and punctuation marks(e.g. ?) and plural forms. These have been removed. 
I haven't removed stop words from the datasets, as some stopwords like I, You etc. can be beneficial for dialog system.
Finally these preprocessed words are tokenized and decapitalized.


In [0]:
# Please comment your code
# These are just common English contractions. There are some of those.

contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]



def preprocess_personality_bot(sentence):
  
  contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
  def expand_contractions(s, contraction_dict=contraction_dict):
    def replace(match):
      return contraction_dict[match.group(0)]
    return contraction_re.sub(replace, s)


  def remove_punctuation(s):
    s = str(s)
    for punct in puncts:
        if punct in s:
            s = s.replace(punct, '')

    return s

  def remove_lemma(s):
    tokens = nltk.word_tokenize(s)
    token_wo_lemma = []
    for w in tokens:
        token_wo_lemma.append(WordNetLemmatizer().lemmatize(w))
    return token_wo_lemma


  prep_sentence = sentence.apply(lambda x:expand_contractions((x.lower())))
  prep_sentence = prep_sentence.apply(lambda x: remove_punctuation(x))
  prep_sentence = prep_sentence.apply(lambda x:remove_lemma((x)))
  
  return prep_sentence


prof_answer = df_prof.Answer
comic_answer = df_comic.Answer
friend_answer = df_friend.Answer

input_df = pd.DataFrame()
input_df['proc_question'] = preprocess_personality_bot(df_prof.Question)
input_df['proc_prof_answer'] = preprocess_personality_bot(df_prof.Answer)
input_df['proc_comic_answer'] = preprocess_personality_bot(df_comic.Answer)
input_df['proc_friend_answer'] = preprocess_personality_bot(df_friend.Answer)

input_df['prof_answer'] = df_prof.Answer
input_df['comic_answer'] = df_comic.Answer
input_df['friend_answer'] = df_friend.Answer


# 2 - Model Implementation

## 2.1. Word Embeddings

*You are required to describe which model was implemented (i.e. Word2Vec with CBOW, FastText with SkipGram, etc.) with justification of your decision *

I have used gensim word2vec with CBOW model to train the model, because it is faster.
Our corpus is a twitter corpus, which does have a lot of infrequent words.
So, skip gram wasn't necessary.
Also, FastText does not show model loss, that was another reason for not choosing fasttext

### 2.1.1. Download Dataset for Word Embeddings



*You are required to describe which data was used with justification of your decision.*

Personality datasets are small in size and hence not suitable for training.
I chose twitter_en.gz dataset as it would contain modern linguals and vocabulary,  and I am going to use similar languals to chat with the bot.
link -- https://github.com/Marsan-Ma/chat_corpus

In [0]:
# Please comment your code
id = '1Fl6bPfe_NUqiShOFaZkU7KVN4xn4cRRS'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('twitter.gz')

### 2.1.2. Data Preprocessing for Word Embeddings

*You are required to describe which preprocessing techniques were used with justification of your decision.*

Instead of using my own preprocessing function(written in section 1.2), I used gensim pre-process function. 
Gensim preprocess step can process the large corpus in reasonable  time.
This preprocessing step tokenizes, decapitalizes and removes punctionation and does not remove stop words, which are important for dialog system.

In [0]:
corpus = []
with gzip.open ('twitter.gz', 'rb') as f:
        for i,line in enumerate (f):
            preprocessed_line = gensim.utils.simple_preprocess(line)
            corpus.append(preprocessed_line)

### 2.1.3. Build Word Embeddings Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

I chose size of embedding to be decently large (150) and 
learning rate 0.03 steadily reduced loss after every iteration.
with smaller learning rate, decrease in loss was slow.
Average length of a sentence in my corpus is 15, that's why I chose window size 10.
After 10 iterations, loss was reduced by 33% of it's initial value and 
it was able to provide similarity of two words quite well. SO, I stopped at 10.


In [0]:
import os
class EpochSaver(CallbackAny2Vec):
    def __init__(self, savedir):
        self.savedir = savedir
        self.epoch = 0

        os.makedirs(self.savedir, exist_ok=True)

    def on_epoch_end(self, model):
        savepath = os.path.join(self.savedir, "w2vmodel_{}_epoch.gz".\
                                format(self.epoch))
        model.save(savepath)
        print(
            "Epoch saved: {}".format(self.epoch + 1),
            "Start next epoch ... ", sep="\n"
            )
        if os.path.isfile(os.path.join(self.savedir, "w2vmodel_{}_epoch.gz".\
                                       format(self.epoch - 1))):
            print("Previous model deleted ")
            os.remove(os.path.join(self.savedir, "w2vmodel_{}_epoch.gz".\
                                   format(self.epoch - 1))) 
        self.epoch += 1
        print("Word2Vec Model loss:", model.get_latest_training_loss())
        
def train_word2vec(iter):

    ### Initialize model ###
    print("Start training Word2Vec model")
    corpus_list = corpus
    w2vmodel = gensim.models.Word2Vec(
        corpus_list,
        size=150, alpha=0.03, min_alpha=0.00025, iter=iter,
        min_count=10, hs=0, negative=10, workers=10,
        window=10, callbacks=[EpochSaver("./")], 
        compute_loss=True
) 

### 2.1.4. Train Word Embeddings Model

In [10]:
# Please comment your code
# train word2vec model
iter = 10
train_word2vec(iter)

Start training Word2Vec model
Epoch saved: 1
Start next epoch ... 
Word2Vec Model loss: 2177288.0
Epoch saved: 2
Start next epoch ... 
Previous model deleted 
Word2Vec Model loss: 4040886.0
Epoch saved: 3
Start next epoch ... 
Previous model deleted 
Word2Vec Model loss: 5681168.0
Epoch saved: 4
Start next epoch ... 
Previous model deleted 
Word2Vec Model loss: 7266022.0
Epoch saved: 5
Start next epoch ... 
Previous model deleted 
Word2Vec Model loss: 8780709.0
Epoch saved: 6
Start next epoch ... 
Previous model deleted 
Word2Vec Model loss: 10216491.0
Epoch saved: 7
Start next epoch ... 
Previous model deleted 
Word2Vec Model loss: 11601287.0
Epoch saved: 8
Start next epoch ... 
Previous model deleted 
Word2Vec Model loss: 12959488.0
Epoch saved: 9
Start next epoch ... 
Previous model deleted 
Word2Vec Model loss: 14307771.0
Epoch saved: 10
Start next epoch ... 
Previous model deleted 
Word2Vec Model loss: 15626345.0


By looking at the loss value, it seems like loss is increasing after getting trained. But this is not as it seems.
This is an representation problem of gensim -->
https://github.com/RaRe-Technologies/gensim/pull/2135

In reality, Loss value after ith iteration = (loss  after ith iteration )- (loss after i-1th iteration)

### 2.1.5. Save Word Embeddings Model

In [0]:
# Please comment your code
'''
Please see 2.1.3
I have a block of code in model build that saves the model at the end of
every epoch
'''

### 2.1.6. Load Word Embeddings Model

In [0]:
# Please comment your code
# restore the model
class EpochSaver(CallbackAny2Vec):
    def __init__(self, savedir):
        self.savedir = savedir
        self.epoch = 0

        os.makedirs(self.savedir, exist_ok=True)

    def on_epoch_end(self, model):
        savepath = os.path.join(self.savedir, "w2vmodel_{}_epoch.gz".\
                                format(self.epoch))
        model.save(savepath)
        print(
            "Epoch saved: {}".format(self.epoch + 1),
            "Start next epoch ... ", sep="\n"
            )
        if os.path.isfile(os.path.join(self.savedir, "w2vmodel_{}_epoch.gz".\
                                       format(self.epoch - 1))):
            print("Previous model deleted ")
            os.remove(os.path.join(self.savedir, "w2vmodel_{}_epoch.gz".\
                                   format(self.epoch - 1))) 
        self.epoch += 1
        print("Word2Vec Model loss:", model.get_latest_training_loss())

w2v_saved_model = gensim.models.Word2Vec.load('./w2vmodel_9_epoch.gz')

In [0]:
index2word_set = set(w2v_saved_model.wv.index2word)

# this function assigns one vector for each answer of personality dataset.
# sentence --> Question
# w2v_saved_model --> the restored model
# num_features --> size of each vector
# index2word_set --> index and word mapping of the corpus
# returns a vector of size 150

def avg_feature_vector(sentence, w2v_saved_model, num_features, index2word_set):

    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in sentence:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, w2v_saved_model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec.tolist()

## 2.2. Seq2Seq model

### 2.2.1. Apply/Import Word Embedding Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
# Please comment your code
# this function returns one-hot encoded question vector and answer vector
# seq_data --> list of Question vector, answer vector tuple
# num_dic --> dictionary made from all the unique words in one personality dataset
# max_input_words_amount --> returns number of tokens/vectors of the question set

def make_batch(seq_data, num_dic, max_input_words_amount):
  input_batch = []
  output_batch = []
  target_batch = []
  
  for seq in seq_data: 
      # Input for encoder cell, convert question to vector

      input_data = get_vectors_q(seq[0], num_dic, max_input_words_amount)  


      # Input for decoder cell, Add '_B_' at the beginning of the sequence data

      output_data = [num_dic[str(['_B_'])]]
      output_data += get_vectors_a(seq[1], num_dic)

      target = [num_dic[str(seq[1])]]
      target.append(num_dic[str(['_E_'])])
      dic_len = len(num_dic)

      # Convert each token vector to one-hot encode data
      input_batch.append(np.eye(dic_len)[input_data])
      output_batch.append(np.eye(dic_len)[output_data])
      target_batch.append(target)

  return input_batch, output_batch, target_batch


# This function is called to make sure program does not fail
# when input is not in corpus. when input is not in vocabulary it returns a 
# list of 150(dimension of vector) zeroes.

def catch_NOV_error(func, handle=lambda e : e, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return np.zeros(shape=(150,)).tolist()

# This function is called 3 times each for one personality(answer_type).
# based on the answer_type, function cooses the answer set and applies 
# word embedding on questions and answers.
# This function returns below parameters.

# unique_words--> unique vectors in the personality dataset. 
# Each vocabulary of question gets a unique vector, each answer gets a unique vector
# We get answer vector by calling avg_feature_vector()
# answer_lookup --> list of all the answer vectors. 
# Will be used to convert the vector to original answer
# num_dic --> dictionary made from unique_words, where each key is the 
# string representation of vector

def prep_data_for_batch(answer_type):
  seq_data = []
  answer_lookup = []
  unique_words = []
  max_input_words_amount = 0
  for index, row in input_df.iterrows():
      tokenized_q = row.proc_question
            
      # we need to tokenise question    
      question_vector = [catch_NOV_error(lambda : w2v_saved_model.wv[i].\
                                         tolist()) for i in tokenized_q]
      
      if answer_type == 'Friend':
        answer_row = row.proc_friend_answer
      elif answer_type == 'Comic':
        answer_row = row.proc_comic_answer
      else:
        answer_row = row.proc_prof_answer
      
      answer_vector = avg_feature_vector(answer_row, w2v_saved_model, 150, index2word_set)
      
      seq_data.append([question_vector, answer_vector])
      answer_lookup.append(answer_vector)
      # add question list and answer list (one element)
      for item in question_vector:
          if item not in unique_words:
              unique_words.append(item)

      if answer_vector not in unique_words:
          unique_words.append(answer_vector)

      # we need to decide the maximum size of input word tokens
      max_input_words_amount = max(len(question_vector), max_input_words_amount)

# adding special tokens in the vocabulary list    
# _B_: Beginning of Sequence
# _E_: Ending of Sequence
# _P_: Padding of Sequence - for different size input
# _U_: Unknown element of Sequence - for different size input

  unique_words = [str(item) for item in unique_words]

  unique_words.append(repr(['_B_']))
  unique_words.append(repr(['_E_']))
  unique_words.append(repr(['_P_']))
  unique_words.append(repr(['_U_']))

  num_dic = {n: i for i, n in enumerate(unique_words)}
  dic_len = len(num_dic)
  
  
### Neural Network Model
  tf.reset_default_graph()

  return unique_words, answer_lookup, seq_data, dic_len, num_dic, max_input_words_amount

# get token index vector of questions and add paddings 
# if the word is shorter than the maximum number of words
def get_vectors_q(sentence, num_dic, max_input_words_amount):
  diff = max_input_words_amount - len(sentence)
  # add paddings if the word(vector) is shorter than the maximum number of words    
  for x in range(diff):
      sentence.append(['_P_'])    
  data = tokens_to_ids(sentence, num_dic)       
  return data

# get token index vector of answer
def get_vectors_a(vector, num_dic):    
  ids = []
  if str(vector) in num_dic:
      ids.append(num_dic[str(vector)])
  else:
      ids.append(num_dic[str(['_U_'])])
  data = ids
  return data


 # convert tokens to index
def tokens_to_ids(vector_list, num_dic):
  ids = [] 
  for vector in vector_list:
      if str(vector) in num_dic:
          ids.append(num_dic[str(vector)])
      else:
          ids.append(num_dic[str(['_U_'])])
  return ids

 


### 2.2.2. Build Seq2Seq Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

This functions build, train and save the seq2seq model for each personality.
Hyper-parameter 0.01 reduces training loss quite faster, 
within first 2 epoch, loss reduces from 6 to 2. 
These seq2seq models are huge in size, so I used epoch(300) only
and got optimal result(loss = 2) by choosing learning rate 0.01

In [0]:
# # Please comment your code

def build_train_save_s2s(personality):

  seq_data = []
  ### Setting Hyperparameters
  unique_words, answer_lookup, seq_data, dic_len, num_dic,\
                       max_input_words_amount = prep_data_for_batch(personality)
  learning_rate = 0.01
  n_hidden = 128

  n_class = dic_len
  n_input = dic_len

  ### Neural Network Model
  tf.reset_default_graph()
  
  # encoder/decoder shape = [batch size, time steps, input size]
  enc_input = tf.placeholder(tf.float32, [None, None, n_input])
  dec_input = tf.placeholder(tf.float32, [None, None, n_input])

  # target shape = [batch size, time steps]
  targets = tf.placeholder(tf.int64, [None, None])
  
  # Encoder Cell
#   with g1.as_default() as g:
  with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                            dtype=tf.float32)
  # Decoder Cell
  with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    # [IMPORTANT] Setting enc_states as inital_state of decoder cell
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)

  seq2seq_model = tf.layers.dense(outputs, n_class, activation=None)

  cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=seq2seq_model, labels=targets))

  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
  ### Training Model
  saver1 = tf.train.Saver(reshape=True)
  
  sess1 = tf.Session()
  sess1.run(tf.initialize_all_variables())
  
  # Generate a batch data
  input_batch, output_batch, target_batch= make_batch(seq_data, num_dic, max_input_words_amount)

  total_epoch = 300

  for epoch in range(total_epoch):
      _, loss = sess1.run([optimizer, cost],
                         feed_dict={enc_input: input_batch,
                                    dec_input: output_batch,
                                    targets: target_batch})
      if epoch % 100 == 0:
          print('Epoch:', '%04d' % (epoch + 1),
                'cost =', '{:.6f}'.format(loss))

  print('Epoch:', '%04d' % (epoch + 1),
        'cost =', '{:.6f}'.format(loss))
  print('Training completed')
  tf.add_to_collection('seq2seq_model', seq2seq_model)
  tf.add_to_collection('enc_input', enc_input)
  tf.add_to_collection('dec_input', dec_input)
  tf.add_to_collection('targets', targets)

  saver1.save(sess1, './s2s_{}_model_new'.format(personality))
  sess1.close()

    


### 2.2.3. Train Seq2Seq Model

In [16]:
build_train_save_s2s('Professional')

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost = 6.357491
Epoch: 0101 cost = 2.279769
Epoch: 0201 cost = 2.288945
Epoch: 0300 cost = 2.370597
Training completed


In [17]:
build_train_save_s2s('Friend')

Epoch: 0001 cost = 6.312863
Epoch: 0101 cost = 2.271080
Epoch: 0201 cost = 2.037329
Epoch: 0300 cost = 1.988876
Training completed


In [18]:
build_train_save_s2s('Comic')

Epoch: 0001 cost = 6.330794
Epoch: 0101 cost = 2.654783
Epoch: 0201 cost = 4.134859
Epoch: 0300 cost = 2.351503
Training completed


### 2.2.4. Save Seq2Seq Model

In [0]:
# models were saved while training in section 2.2.

In [0]:
# downlaod the models to save in google drive
files.download('w2vmodel_9_epoch.gz')
s2s_files = [files.download(f) for f in os.listdir('.') if re.match(r's2s*', f)]

### 2.2.5. Load Seq2Seq Model

In [0]:
# Please comment your code
# This function loads the saved s2s models and get the saved tensors 
# then, it calls prep_data_for_batch to get unique_words, answer_lookup, num_dic 
# for that personality
# Finally it calls response() method to get the answer.
def answer(sentence, conv_type):
  tf.reset_default_graph()
  if conv_type == 'Friend':
    model_name = 'Friend'
  elif conv_type == 'Comic':
    model_name = 'Comic'
  else:
    model_name = 'Professional'
  
  answer_lookup = []
  unique_words = []
  num_dic = {}
  max_input_words_amount = 0
  saver = tf.train.import_meta_graph("./s2s_{}_model_new.meta".format(model_name))
  sess =  tf.Session()

 
  # Restore (Load) the model
  saver.restore(sess, "./s2s_{}_model_new".format(model_name))
  
  enc_input = tf.get_collection('enc_input')[0]
  dec_input = tf.get_collection('dec_input')[0]
  targets = tf.get_collection('targets')[0]
  seq2seq_model = tf.get_collection('seq2seq_model')[0]

  unique_words, answer_lookup, seq_data, dic_len, num_dic, max_input_words_amount = prep_data_for_batch(model_name)
  
  bot_response = response(sentence, conv_type, sess, unique_words, answer_lookup, num_dic, seq2seq_model, enc_input, dec_input, targets, max_input_words_amount )
  
  sess.close()
  return bot_response


# This function runs the restored model to get the prediction of user input
# and returns the answer of user question
# sentence --> list of question vector and answer vector tuple, 
# Since, we do not know the answer at the beginning, answer vector is '_U_'

def response(sentence, conv_type, sess,  unique_words, answer_lookup, num_dic, seq2seq_model, enc_input, dec_input, targets, max_input_words_amount):
  answer_index = 0
  seq_data = [sentence, ['_U_']]

  input_batch, output_batch, target_batch = make_batch([seq_data], num_dic, max_input_words_amount)

  prediction = tf.argmax(seq2seq_model, 2)

  result = sess.run(prediction,
                    feed_dict={enc_input: input_batch,
                               dec_input: output_batch,
                               targets: target_batch})

  
  decoded = [unique_words[i] for i in result[0]]
  # in case, program malfunctions and not able to find the answer 
  # in answer vector list, instead of failing, it will return 'Oops'
  try:
    answer_index = (answer_lookup.index(eval(decoded[0])))
  except:
    response = "Oops"
  
  if conv_type == 'Friend':
    col_name = 'friend_answer'
  elif conv_type == 'Comic':
    col_name = 'comic_answer'
  else:
    col_name = 'prof_answer'
    
  # gets the answer by looking up the personality datadrame by answer_index
  response = input_df.iloc[answer_index][col_name]

  return response



# 3 - Evaluation (Running chatbot)

## 3.1. Start chatting

## 3.2. Change Personality

*Explain how to change personality (What is the command for changing personality?). *

User can change the personality by entering command 'change personality'
Bot then asks for user's choice of personality
User can enter 'Friend or 'Comic' to enter these personality modes,
Any other input will bring him to professional bot.

In [0]:
# Please comment your code


def change_personality(input):
  if input == 'Friend':
    print('Hey Friend')
    personality = 'Friend'
  elif input == 'Comic':
    print('I am funny, talk to me')
    personality = 'Comic'
  else:
    print("Let's talk business")
    personality = 'Professional'
  return personality



## 3.3. Save chat log

Once user enters end chat command 'end_conv',
Bot gives the option of saving chat file. If user says Y, 
The chat log is automatically saved.

In [0]:
# supress showing information like 'model is restored'
import logging
import logging.config
logging.config.dictConfig({
    'version': 1,
    'disable_existing_loggers': True,
})

logger = logging.getLogger("answer")

In [0]:
# Please comment your code
# This function takes input from user and generates bot response
# At the end of the conversation user can choose to save the chat log

def call_bot():
  if os.path.exists('chat_log.txt'):
    os.remove('chat_log.txt')
  personality = 'Professional'
  user_sentence = 'start chat'
  while user_sentence!= 'end_conv':
    user_sentence = input('User:')
    # default personality is professional
    if user_sentence == 'change personality':
      print('Give me your choice')
      personality_input = input("User:")
      personality = change_personality(personality_input)
      user_sentence = input("User:")
    if user_sentence == 'end_conv':
      print("Press Y if you want a chat log")
      user_sentence = input('User:')
      if user_sentence == 'Y':
        files.download('chat_log.txt')
      print("Good talk, bye for now")
      break  
    my_tokenized_q = nltk.word_tokenize(user_sentence)
    my_question_vectors =[catch_NOV_error(lambda:w2v_saved_model.wv[i.lower()].\
                                          tolist()) for i in my_tokenized_q]
    # calls answer() to run the restored model and get the answer
    bot_answer = answer(my_question_vectors, personality)
    print('Bot:', bot_answer)
    with open("chat_log.txt", "a") as f:
      f.write("{} --> {}".format(user_sentence, bot_answer) + "\r\n")
      
      





## 3.4. End chatting

In [0]:
# Please comment your code
# In 3.3. section, end chat command is encapsulated in the function
# end chat command is 'end_conv'

## 3.5. Execute program

***Please make sure your program  is running properly.***

***Functions for downloading (from Google Drive) and loading models (both word embeddings and Seq2Seq) need to be called!*** 


Instruction for execution:
1. Execute 1.1. section to execute import statements
2. Execute 1.2 section to pre-process personality datasets
3. Execute 3.5 section to restore the models
4. Execute 2.1.6 to restore word2vec model
5. Execute 2.2.1 which defines the function to apply word2vec model
6. Execute 2.2.5 to load the seq2seq models
7. Execute 3.2 to define change personality function
8. Execute 3.3 to define chat function
9. Execute 3.5.2 to start chatting

In [0]:
# Code to download saved models into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1aHHX9m68AmqmPl4hsKSfhjL3QwLXId0L'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('s2s_Comic_model_new.data-00000-of-00001')

id = '1OTWwxWHaW7_XuxTQpCuOZFdLNNcrvyQo'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('s2s_Comic_model_new.index')

id = '1-MgEsDUGMQaqLwdGbXOD6zuFnQzSi-k3'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('s2s_Comic_model_new.meta')

id = '1Q9XVc05BTHE9Mi7HdYnEL-_V6ddftWjF'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('s2s_Professional_model_new.data-00000-of-00001')

id = '1jDVrOqe5buIYjCRSGsnQQdmeli5xB9_Q'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('s2s_Professional_model_new.index')

id = '1XkT2TEKVMEE7EhgCArQNZd_Pt_htb3pu'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('s2s_Professional_model_new.meta')

id = '1l90mP9oIaeSN8fvqS4IRewHD7RDlbjes'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('s2s_Friend_model_new.data-00000-of-00001')

id = '1NHFSEz65rVNQTOSqraMnK0s41r0F_jy-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('s2s_Friend_model_new.index')

id = '1M2XhYWN7v7BLD4QmDV5dHz_Zd5tA6eAs'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('s2s_Friend_model_new.meta')

id = '1XivBd-pvHo4Cfendlg7CpBLGqbPNaFdD'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('w2vmodel_9_epoch.gz')


### 3.5.1. Execute program - training mode

*Please include lines to train the bot.*

In [0]:
# Please comment your code

# we can train seq2seq model by executing 2.2.2
# we can train word2vec model by executing 2.1.4

### 3.5.2. Execute program - chatting mode

*Please include lines to start chatting with the bot.*

In [14]:
# Please comment your code
print("Let's start talking")
call_bot()

Let's start talking
User:how are you
Bot: I don't have family.
User:are you happy
Bot: I don't have family.
User:do you know other bots
Bot: That's a biological concept that doesn't apply to me.
User:you are stupid
Bot: That's not me, but hello.
User:i miss you
Bot: I don't have family.
User:NLP assignment is over
Bot: I don't have family.
User:it was good
Bot: I'm afraid I'm not musically inclined.
User:you are sweet
Bot: Let's keep things professional.
User:you are giving wrong answers
Bot: Goodbye.
User:do you like me
Bot: I don't have family.
User:who is god
Bot: I don't have family.
User:life is short
Bot: I hope you're able to get some rest soon.
User:i hate my life
Bot: I do what I can.
User:i love to travel
Bot: I aim for efficiency.
User:i love my mother
Bot: I'm afraid I'm not musically inclined.
User:Do you have family
Bot: I aim for efficiency.
User:change personality
Give me your choice
User:Comic
I am funny, talk to me
User:how are you
Bot: Those who can, do. Those who ca

## Object Oriented Programming codes here

*If you have multiple classes use multiple code snippets to add them.*